### Instalação de módulos

In [100]:
pip install numpy -q

In [101]:
pip install pandas -q

In [102]:
pip install scikit-learn -q

### Importando bibliotecas

In [103]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import seaborn as sns

## Tratamento de dados

### Carregando os dados

In [104]:
df = sns.load_dataset('iris')
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

### Ajustando os dados

In [105]:
df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True) # substituindo o nome das classes

df.columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Class"] # renomeando as colunas

labels = df["Class"].values # separando as classes do resto do dataset
df.drop("Class", axis=1, inplace=True) # removendo as classes do resto dos dados

<ipython-input-105-bec1708fd776>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True) # substituindo o nome das classes
<ipython-input-105-bec1708fd776>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["species"].repl

### Transformando os dados em uma array

In [106]:
dados = df.to_numpy()
dados

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

## Clustering

### Inicialização da matriz de pertinência

A matriz de pertinência é inicializada aleatoriamente $u_{ijk}(i=1,...c$; $j=1,...,p$ e $k=1,...,n)$ do objeto $k$ pertencente ao grupo $C_i$ para a variável $j$ tal que:
- $u_{ijk} \in [0,1]$;
- $0 < \sum_{k=1}^nu_{ijk} < n$;
- $\sum_{i=1}^c\sum_{j=1}^pu_{ijk} = 1$ para todo $k \in \Omega$.

In [107]:
def inicializacao_matriz_pertinencia(num_amostras, num_clusters, num_variaveis):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters, num_variaveis) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

### Atualização dos centroides

Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:

### $y_{ij} = \frac{\sum_{k=1}^n(u_{ijk})^mx_{kj}}{\sum_{k=1}^n(u_{ijk})^m}$

In [108]:
def atualizacao_centroides(dados, matriz_pertinencia, m):
    num_clusters = matriz_pertinencia.shape[1]
    num_variaveis = dados.shape[1]

    matriz_pertinencia_m = matriz_pertinencia ** m
    centroides = np.zeros((num_clusters, num_variaveis)) # inicializando os centroides
    # precisa porque como são duas dimensões não dá pra fazer a vetorização como no FCM

    for j in range(num_variaveis):  # para cada variável
        for i in range(num_clusters):  # para cada cluster
            numerador = np.sum(matriz_pertinencia_m[:, i, j].reshape(-1, 1) * dados[:, j].reshape(-1, 1)) # reshape usado para garantir que não tenha problemas na operação
            denominador = np.sum(matriz_pertinencia_m[:, i, j])
            centroides[i, j] = numerador / denominador

    return centroides

### Atualização da matriz de pertinência

Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:

### $u_{ijk} = [\sum_{a=1}^c\sum_{b=1}^p(\frac{d_{ijk}}{d_{abk}})^\frac{1}{m-1}]^{-1}$

In [109]:
def atualizacao_matriz_pertinencia(dados, centroides, m):
    num_amostras, num_clusters, num_variaveis = dados.shape[0], centroides.shape[0], dados.shape[1] # informações importantes
    matriz_distancias = np.zeros((num_amostras, num_clusters, num_variaveis)) # inicializando a matriz de distâncias

    for k in range(num_variaveis): # para cada variável
        matriz_distancias[:, :, k] = np.square(dados[:, np.newaxis, k] - centroides[:, k]) # distância entre cada ponto de dados e cada centroide para cada variável

    matriz_pertinencia_atualizada = np.zeros((num_amostras, num_clusters, num_variaveis)) # inicializando a matriz de pertinência multivariada
    for k in range(num_variaveis): # precisa fazer pelo mesmo motivo da atualização de centroides
        #cCalcula a nova pertinência usando uma soma direta (fórmula vetorizada)
        dist_inv = np.fmax(matriz_distancias[:, :, k], np.finfo(np.float64).eps)  # evita divisões por zero
        matriz_pertinencia_atualizada[:, :, k] = 1 / dist_inv ** (1 / (m - 1)) # cálculo da pertinência
        matriz_pertinencia_atualizada[:, :, k] /= np.sum(1 / dist_inv ** (1 / (m - 1)), axis=1, keepdims=True) # normalização dos graus de pertinência para garantir as 3 regras

    return matriz_pertinencia_atualizada


### Multivariate Fuzzy C-Means

Ações:
1. Inicialização da matriz de pertinência
2. Atualização dos centroides
3. Atualização da matriz de pertinência

Critérios de parada:
1. Número máximo de iterações atingido
2. Pouca diferença (erro) entre as matrizes de pertinência de iterações consecutivas

In [110]:
def mfcm(dados, num_clusters, m=2, max_iter=1000, erro=1e-5):
    num_amostras, num_variaveis = dados.shape # informações importantes para evitar aumentar o número de parâmetros
    matriz_pertinencia = inicializacao_matriz_pertinencia(num_amostras, num_clusters, num_variaveis)

    for _ in range(max_iter):  # critério de parada por número de iterações
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, m)

        # critério de convergência
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro:
            break
        matriz_pertinencia = nova_matriz_pertinencia

    return centroides, matriz_pertinencia

### Índice de Rand Ajustado (IRA)

In [111]:
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

### Agregação dos graus de pertinência

### $\delta_{ik} = \sum_{j=1}^pu_{ijk}$

In [112]:
def calcular_delta(matriz_pertinencia): # para que os labels preditos sejam unidimensionais
    delta = np.sum(matriz_pertinencia, axis=2)
    return delta

### Simulação de Monte Carlo

In [113]:
def simulacao_monte_carlo(dados, labels, num_clusters, num_trials):
    indices_rand = []
    for _ in range(num_trials):
        centroides, matriz_pertinencia = mfcm(dados, num_clusters)
        delta = calcular_delta(matriz_pertinencia) # etapa adicional quando comparado ao FCM
        predicted_labels = np.argmax(delta, axis=1)
        idx_rand = indice_rand(labels, predicted_labels)
        indices_rand.append(idx_rand)
    mean_rand_index = np.mean(indices_rand)
    std_rand_index = np.std(indices_rand)
    return mean_rand_index, std_rand_index

### Definição de parâmetros e execução do método

In [115]:
num_clusters = 3
num_trials = 100
media_indice_rand, dp_indice_rand = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)

print(f"Monte Carlo MFCM Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {media_indice_rand:.4f}") # resultado esperado: 0.1786
print(f"Standard Deviation of Rand Index: {dp_indice_rand:.4f}") # esperado quase ou nenhum desvio padrão

Monte Carlo MFCM Clustering Results (100 trials)
Mean Rand Index: 0.4158
Standard Deviation of Rand Index: 0.1978
